# Lesson 3

In [1]:
#import stuff
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [7]:
%matplotlib inline
import os
import utils; reload(utils)
from utils import *
from __future__ import print_function, division

In [10]:
#setting the path to the data and the models
path = 'data/dogscats/'
model_path = path + 'model/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [8]:
model = vgg_ft(2)

In [11]:
model.load_weights(model_path+'finetune6.h5')

IOError: Unable to open file (Unable to open file: name = 'data/dogscats/model/finetune6.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [13]:
layers = model.layers

In [16]:
#get the index of the last convlution layer
last_conv_idx = [index for index, layer in enumerate(layers)
                     if type(layer)==Convolution2D][-1]

In [17]:
last_conv_idx

30

In [19]:
#get all the conv layers now that we have the last idx and create a model from it
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)

In [59]:
#get the fully connected layers, we need them for their weights
fc_layers = layers[last_conv_idx+1:]

In [21]:
batch_size = 64

In [23]:
batches = get_batches(path+'train/', batch_size=batch_size)

Found 23000 images belonging to 2 classes.


In [24]:
val_batches = get_batches(path+'valid/', batch_size=batch_size)

Found 2000 images belonging to 2 classes.


In [25]:
train_classes = batches.classes
val_classes = val_batches.classes
train_labels = onehot(train_classes)
val_classes = onehot(val_classes)

In [26]:
??model.predict_generator

In [31]:
train_features = conv_model.predict_generator(batches, batches.N)

In [34]:
save_array(model_path+'train_features_conv.bc', train_features)

In [37]:
val_features = conv_model.predict_generator(val_batches, val_batches.N)

In [38]:
save_array(model_path+'val_features_conv.bc', val_features)

In [55]:
#optimiser for the fully connected model lr will be very low as we are training a fully connected model
opt = RMSprop(lr=0.00001, rho=0.7)

In [56]:
#method to get weights
#note : I am fairly sure that this is the rescaling of weights. Since Keras automatically does that we might not need to divide by 2 here. 
def proc_weights(layer): return [o/2 for o in layer.get_weights()]

In [57]:
def get_fc_model():
    model = Sequential([
            MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
            Flatten(),
            Dense(4096, activation='relu'),
            Dropout(0.),
            Dense(4096, activation='relu'),
            Dropout(0.),
            Dense(2, activation='softmax')
        ])
    
    for l1, l2 in zip(model.layers, fc_layers): l1.set_weights(proc_weights(l2))
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [60]:
fc_model = get_fc_model()

In [ ]:
fc_model.fit(train_fetures, train_labels, batch_size=batch_size, epochs=8, validation_data=(val_features, val_labels))

In [ ]:
fc_model.save_weights(model_path+'no_dropout.h5')